# ლექცია #3

## კოდების გაშვებისთვის საჭირო ბიბლიოთეკების იმპორტები

ამ ნოუთბუქის გაშვებამდე არ დაგავიწყდეთ საჭირო დამოკიდებულებების (dependencies) დაინსტალირება, რომლებიც მოცემულია `requirements.txt` ფაილში. მარტივად, ტერმინალიდან გაუშვით:

```bash
pip install -r requirements.txt
```

In [17]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

## მონაცემების ჩატვირთვა

ამ ლექციაში გამოვიყენებთ Swarm Behaviour (ხროვის, დაჯგუფების ქცევის) მონაცემებს, რომლებიც შეგროვებულია ახალი სამხრეთ უელსის უნივერსიტეტის მიერ.

**Flocking behaviour** ნიშნავს ფრინველების, მწერების, თევზების ან სხვა ცხოველების ჯგუფების გადაადგილებას ერთმანეთთან ახლოს. მათ შეუძლიათ ჯგუფურად გადაადგილება ერთი და იგივე სიჩქარით, თუმცა ერთმანეთთან შეჯახების გარეშე.

**Align** ნიშნავს მათ მოძრაობას ერთი მიმართულებით.

**Group** ნიშნავს თუ რამდენად არიან დაჯგუფებულები.

მათ შექმნეს გამოკითხვა, სადაც ადამიანებს უწევდათ დაკვირვებოდნენ ხროვის/დაჯგუფების ქცევას და შესაბამისად ეთქვათ იყო თუ არა მოცემული ქცევა სურათზე Flocking, Aligned, Grouped.

მახასიათებლებია $x_m$, $y_m$, ($X$, $Y$) პოზიცია თითოეული boid-ის (Boids არის ხელოვნური სიცოცხლის პროგრამა, შემუშავებული კრეიგ რეინოლდსის მიერ 1986 წელს, რომელიც სიმულაციას უწევს ფრინველთა ქცევას და მასთან დაკავშირებულ ჯგუფურ მოძრაობას), $xVel_n$, $yVel_n$ როგორც სიჩქარის ვექტორი, $xA_m$, $yA_m$ როგორც განლაგების (alignment) ვექტორი, $xS_m$, $yS_m$ როგორც განცალკევების (separation) ვექტორი, $xC_m$, $yC_m$ როგორც გაერთიანების (cohesion) ვექტორი, $nAC_m$, როგორც boid-ების რაოდენობა გასწორების/თანმიმდევრობის რადიუსში და $nS_m$, როგორც boid-ების რაოდენობა განცალკევების რადიუსში. ეს ატრიბუტები მეორდება ყველა m boid-ისთვის, სადაც $m=1, ... , 200$. ასევე, ეტიკეტები არის ორობითი, 1 მიუთითებს Flocking-ზე, Gouped-ზე და Aligned-ზე, ხოლო 0 აღნიშნავს Not Flocking, Not Gruped და Not Aligned.

დამატებითი ინფორმაციისთვის შეგიძლიათ ნახოთ ამ გამოკითხვის [დაარქივებული სქრინშოთები](https://unsw-swarm-survey.netlify.app/Files/Screenshots.pdf).

რადგანაც მონაცემები მოცემულია CSV ფორმატით, მარტივად შეგვიძლია მისი წაკითხვა `pandas` ბიბლიოთეკით.

In [18]:
df = pd.read_csv("./swarm_behaviour.csv")
df.head()

,x1,y1,xVel1,yVel1,xA1,yA1,xS1,yS1,xC1,yC1,...,yVel200,xA200,yA200,xS200,yS200,xC200,yC200,nAC200,nS200,Swarm_Behaviour
0,562.05,-0.62,-10.70,-4.33,0.00,0.00,0.0,0.0,0.00,0.00,...,-15.15,0.00,0.00,0.00,0.00,0.00,0.0,28,0,0.0
1,175.66,-57.09,2.31,-2.67,0.00,0.00,0.0,0.0,0.00,0.00,...,-3.48,0.00,0.00,0.00,0.00,0.00,0.0,4,0,0.0
2,200.16,-320.07,4.01,-6.37,0.00,0.00,0.0,0.0,0.18,-0.26,...,-9.38,0.00,0.00,0.00,0.00,-0.11,-0.3,15,1,0.0
3,316.99,-906.84,0.85,9.17,-0.17,1.03,0.0,0.0,0.00,0.00,...,10.39,-0.26,1.01,0.00,0.00,0.00,0.0,16,0,0.0
4,1277.68,908.54,-2.02,8.23,-1.00,1.00,0.0,0.0,0.00,0.00,...,13.91,-1.00,0.00,3.21,15.67,0.00,0.0,12,0,0.0


In [19]:
df["Swarm_Behaviour"].value_counts()

Swarm_Behaviour
0.0    15355
1.0     7954
Name: count, dtype: int64

In [20]:
df["Swarm_Behaviour"].sum() / len(df)

0.34124158050538417

როგორც ვხედავთ, ასე თუ ისე არადაბალანსირებული ეტიკეტები გვაქვს, თუმცა ამ ეტაპზე ამის აღმოსაფხვრელად არაფერს მოვიმოქმედებთ.

სანამ მონაცემების დამუშავებაზე გადავალთ აუცილებელია ვნახოთ, რომელიმე სვეტში მონაცემი ხომ არ არის გამოტოვებული.

In [21]:
df.isna().sum().sum()

0

## მონაცემების სატრენინგო, ვალიდაციის და სატესტო ნაწილებად დაყოფა

იმისთვის, რომ მონაცემები ისე გავყოთ სატრენინგო, ვალიდაციის და სატესტო ნაწილებად, რომ შევინარჩუნოთ ეტიკეტში 1-იანებისა და 0-იანების თანაფარდობა, შეგვიძლია გამოვიყენოთ `scikit-learn`. თავდაპირველად მონაცემებს ვყოფთ 2 ნაწილად: სატრენინგო და სამომავლოდ ვალიდაციისა და სატესტო მონაცემებად. ამ მაგალითში სატრენინგოდ ავიღეთ მთლიანი მონაცემების 80%.

In [22]:
X_train, X_remain, y_train, y_remain = train_test_split(
    df.drop(columns=["Swarm_Behaviour"]),
    df["Swarm_Behaviour"],
    train_size=0.8,
    random_state=1,
    stratify=df["Swarm_Behaviour"],
)


დარჩენილი 20% შეგვიძლია 2 ტოლ ნაწილად გავყოთ და გამოვიყენოთ, როგორც ვალიდაციის და სატესტო მონაცემები.

In [23]:
X_valid, X_test, y_valid, y_test = train_test_split(
    X_remain,
    y_remain,
    test_size=0.5,
    random_state=1,
    stratify=y_remain,
)

In [24]:
print("სატრენინგო მონაცემების ზომა:", X_train.shape, y_train.shape)
print("ვალიდაციის მონაცემების ზომა:", X_valid.shape, y_valid.shape)
print("სატესტო მონაცემების ზომა:", X_test.shape, y_test.shape)

სატრენინგო მონაცემების ზომა: (18647, 2400) (18647,)
ვალიდაციის მონაცემების ზომა: (2331, 2400) (2331,)
სატესტო მონაცემების ზომა: (2331, 2400) (2331,)


In [25]:
print(
    "1-იანების თანაფარდობა მთელ სატრენინგო მონაცემებზე:", y_train.sum() / len(y_train)
)
print(
    "1-იანების თანაფარდობა მთელ ვალიდაციის მონაცემებზე:", y_valid.sum() / len(y_valid)
)
print("1-იანების თანაფარდობა მთელ სატესტო მონაცემებზე:", y_test.sum() / len(y_test))

1-იანების თანაფარდობა მთელ სატრენინგო მონაცემებზე: 0.3412345149353783
1-იანების თანაფარდობა მთელ ვალიდაციის მონაცემებზე: 0.3410553410553411
1-იანების თანაფარდობა მთელ სატესტო მონაცემებზე: 0.3414843414843415


## მონაცემების სკალირება

"მონაცემთა გაჟონვის" თავიდან არიდების მიზნით, სკალირებას ვახდენთ მხოლოდ სატრენინგო მონაცემებზე, ანუ `StandardScaler` გამოთვლის საშუალოს და სტანდარტულ გადახრას მხოლოდ სატრენინგო მონაცემებზე და მიღებული შედეგებით შეგვიძლია უკვე ვალიდაციის და სატესტო მონაცემების სკალირება მოვახდინოთ.

In [26]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)


## K უახლოესი მეზობლის (KNN) კლასიფიკატორის ტრენინგი

სამწუხაროდ, არ არსებობს რაიმე მარტივი გზა იმისთვის, რომ ვიპოვოთ ოპტიმალური K, ანუ მეზობლების რაოდენობა. ამისთვის გვიწევს ვნახოთ განსხვავებულ K-ებზე როგორ იმუშავებს მოდელი და იქიდან ამოვარჩიოთ ოპტიმალური რიცხვი, თუმცა ეს პროცესი შესაძლოა ხანგრძლივი და გამოსათვლელად, რესურსების მხრივ, რთული იყოს. ამიტომაც 2 წესი არსებობს:

1. K უნდა იყოს კენტი თუ კლასების (განსხვავებული ეტიკეტების) რაოდენობა არის ლუწი.
2. $K = \sqrt{n}$, სადაც $n$ არის მონაცემთა რაოდენობა სასწავლო მონაცემებში.

შესაბამისად, ჩვენ გამოვიყენებთ ორივე წესს.

In [27]:
len(X_train) ** 0.5

136.55401861534503

### KNN-ის ტრენინგი ყველა მახასიათებლის გამოყენებით

In [28]:
knn = KNeighborsClassifier(n_neighbors=137)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=137)

### KNN-ის ტრენინგი შემცირებული რაოდენობის მახასიათებლებით

როგორც ვისწავლეთ, PCA-ის გამოყენება შეგვიძლია, რომ შევამციროთ მახასიათებლების რაოდენობა. შესაბამისად, რომ არ მოხდეს "მონაცემების გაჟონვა", შემცირების სწავლა უნდა მოხდეს მხოლოდ და მხოლოდ სატრენინგო მონაცემებზე.

In [29]:
pca = PCA(n_components=1000, random_state=1)
pca.fit(X_train_scaled)

PCA(n_components=1000, random_state=1)

In [30]:
X_train_reduced = pca.transform(X_train_scaled)
X_valid_reduced = pca.transform(X_valid_scaled)
X_test_reduced = pca.transform(X_test_scaled)

In [31]:
knn_reduced = KNeighborsClassifier(n_neighbors=137)
knn_reduced.fit(X_train_reduced, y_train)

KNeighborsClassifier(n_neighbors=137)

## სიზუსტის შედარებითი ანალიზი სატრენინგო და ვალიდაციის მონაცემებზე

იმისთვის, რომ გავიგოთ ჩვენი მოდელის რამდენად კარგად მუშაობს, შეგვიძლია გამოვთვალოთ სხვადასხვა შეფასების მეტრიკა.

**აკურატულობა** (**accuracy**) არის მეტრიკა, რომელიც ზომავს რამდენად ხშირად ახდენს ML მოდელი შედეგის სწორად პროგნოზირებას:

$$
accuracy = \frac{correct \ predictions}{all \ predictions}
$$

თუ გვაქვს არადაბალანსირებული კლასები, აკურატულობა ნაკლებად სასარგებლოა, რადგან ის თანაბარ წონას ანიჭებს მოდელის უნარს, იწინასწარმეტყველოს ყველა კატეგორია.

ბინარულ კლასიფიკაციაში არის ორი შესაძლო სამიზნე კლასი, რომლებიც, როგორც წესი, ეტიკეტირებულია როგორც "დადებითი" და "უარყოფითი" ან "1" და "0". შესაბამისად, ჩვენ შეგვიძლია ვიყოთ "მართალი" და "მცდარი" ორი განსხვავებული გზით. სწორი პროგნოზები მოიცავს ე.წ. ნამდვილ პოზიტივებს (true positives - TP) და ნამდვილ უარყოფითებს (true negatives - TN). მოდელის შეცდომებს მიეკუთვნება ე.წ ცრუ დადებითები (false positives - FP) და ცრუ უარყოფითები (false negatives - FN).

ამ ოთხივე მნიშვნელობის წარმოდგენა მარტივად შეგვიძლია დაბნეულობის მატრიცაში (confusion matrix):

![Confusion Matrix](https://images.datacamp.com/image/upload/v1701364260/image_5baaeac4c0.png)

შესაბამისად, აკურატულობის ფორმულის ჩაწერა შეგვიძლია შემდეგნაირად:

$$
accuracy = \frac{TP + TN}{TP + TN + FP + FN}
$$

**სიზუსტე** (**precision**) არის მეტრიკა, რომელიც ზომავს რამდენად ხშირად ახდენს ML მოდელი დადებითი კლასის სწორად პროგნოზირებას:

$$
precision = \frac{TP}{TP + FP}
$$

**გახსენება** (**recall**) არის მეტრიკა, რომელიც ზომავს რამდენად ხშირად განსაზღვრავს ML მოდელი დადებით შემთხვევებს (true positives) სწორად მონაცემთა ნაკრების ყველა რეალური დადებითი ნიმუშიდან:

$$
recall = \frac{TP}{TP + FN}
$$

**F1** ქულა არის სიზუსტისა და გახსენების ჰარმონიული საშუალო საზომი. სიზუსტისა და გახსენების ფარდობითი წვლილი F1 ქულაში თანაბარია:

$$
F1 = 2 \times \frac{precision \times recall}{precision + recall}
$$

In [32]:
# index = pd.MultiIndex.from_tuples(
#     [
#         ("KNN", "Train"),
#         ("KNN", "Validation"),
#         ("KNN reduced", "Train"),
#         ("KNN reduced", "Validation"),
#     ],
#     names=["model", "set"],
# )

# metrics = pd.DataFrame(
#     {
#         "accuracy": [
#             accuracy_score(y_train, knn.predict(X_train_scaled)),
#             accuracy_score(y_valid, knn.predict(X_valid_scaled)),
#             accuracy_score(y_train, knn_reduced.predict(X_train_reduced)),
#             accuracy_score(y_valid, knn_reduced.predict(X_valid_reduced)),
#         ],
#         "precision": [
#             precision_score(y_train, knn.predict(X_train_scaled)),
#             precision_score(y_valid, knn.predict(X_valid_scaled)),
#             precision_score(y_train, knn_reduced.predict(X_train_reduced)),
#             precision_score(y_valid, knn_reduced.predict(X_valid_reduced)),
#         ],
#         "recall": [
#             recall_score(y_train, knn.predict(X_train_scaled)),
#             recall_score(y_valid, knn.predict(X_valid_scaled)),
#             recall_score(y_train, knn_reduced.predict(X_train_reduced)),
#             recall_score(y_valid, knn_reduced.predict(X_valid_reduced)),
#         ],
#         "f1": [
#             f1_score(y_train, knn.predict(X_train_scaled)),
#             f1_score(y_valid, knn.predict(X_valid_scaled)),
#             f1_score(y_train, knn_reduced.predict(X_train_reduced)),
#             f1_score(y_valid, knn_reduced.predict(X_valid_reduced)),
#         ],
#     },
#     index=index,
# )
# metrics

ზოგადად, საბოლოო მოდელის შერჩევა ხდება ვალიდაციის მონაცემებზე და არა სატესტო მონაცემებზე. რადგანაც თუ სატესტო მონაცემებზე დაყრდნობით ავირჩევთ საბოლოო მოდელს, ეს "მონაცემთა გაჟონვის" ერთ-ერთი გამოხატულება იქნება. ზემოთ მოცემული შედეგებიდან, ცოტათი რთულია იმის განსაზღვრა თუ რომელი სჯობს, რადგანაც თითქმის თანაბარი ქულებია. ცოტათი მაღალი F1 ქულა აქვს KNN-ს შემცირებულ განზომილებაზე და ამიტომაც ეს ავარჩიოთ ჩვენს საბოლოო მოდელად.

შესაბამისად, გამოვთვლით მეტრიკებს სატესტო მონაცემებზეც:

In [33]:
pd.DataFrame(
    {
        "accuracy": [accuracy_score(y_test, knn_reduced.predict(X_test_reduced))],
        "precision": [precision_score(y_test, knn_reduced.predict(X_test_reduced))],
        "recall": [recall_score(y_test, knn_reduced.predict(X_test_reduced))],
        "f1": [f1_score(y_test, knn_reduced.predict(X_test_reduced))],
    }
)

,accuracy,precision,recall,f1
0,0.844273,0.764994,0.785176,0.774954
